In [76]:
import torch
from torchvision import transforms
from diffusers import AutoencoderKL
from PIL import Image
import requests

In [77]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
# display(image)

In [78]:
image = transforms.Resize(128)(image)
image = transforms.ToTensor()(image).unsqueeze(0)
model = AutoencoderKL().to('cuda')
out = model.encode(image.to('cuda')).latent_dist

In [103]:
out.sample()

tensor([[[[ 6.5473e-01,  1.7594e+00,  1.3192e+00,  ...,  1.9767e-01,
            1.5611e+00, -1.0620e+00],
          [-1.1411e+00,  6.4325e-01,  3.2717e-01,  ..., -1.1717e+00,
           -4.1933e-01,  2.9469e-01],
          [-1.2109e+00, -8.6480e-01, -7.5517e-01,  ..., -8.9837e-01,
           -3.2185e-01, -5.3190e-01],
          ...,
          [-2.2421e-01,  1.6826e+00,  7.2779e-01,  ...,  4.0762e-01,
           -2.6253e-01, -6.4070e-01],
          [-1.8365e+00, -4.4094e-01,  8.6749e-02,  ..., -9.6653e-01,
           -5.5674e-01, -1.0174e+00],
          [ 5.8822e-01,  7.1620e-02,  4.5480e-02,  ..., -7.5032e-01,
           -1.4659e-01,  6.3315e-01]],

         [[ 7.4479e-02, -1.4564e+00, -1.6481e+00,  ..., -7.8668e-01,
            6.9922e-01,  4.8016e-01],
          [ 8.5038e-04,  2.4102e-01,  1.4335e+00,  ..., -1.8494e-01,
           -6.2167e-01,  2.8094e-01],
          [-3.1910e-01, -1.4309e+00,  6.7293e-01,  ...,  6.6274e-01,
           -7.0209e-01,  6.3646e-01],
          ...,
     

In [5]:
from transformers import AutoTokenizer, CLIPTextModel

model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

inputs = tokenizer(["a photo of a cat", "a photo of a dog"], padding=True, return_tensors="pt")

outputs = model(**inputs)
last_hidden_state = outputs.last_hidden_state
pooled_output = outputs.pooler_output  # pooled (EOS token) states

Some weights of the model checkpoint at openai/clip-vit-base-patch32 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.4.layer_norm1.bias', 'vision_model.encoder.layers.0.mlp.fc2.weight', 'vision_model.encoder.layers.1.self_attn.out_proj.weight', 'vision_model.encoder.layers.10.mlp.fc2.weight', 'vision_model.encoder.layers.9.self_attn.out_proj.weight', 'vision_model.encoder.layers.0.mlp.fc1.bias', 'vision_model.encoder.layers.11.self_attn.v_proj.weight', 'vision_model.encoder.layers.1.self_attn.k_proj.weight', 'vision_model.encoder.layers.6.self_attn.out_proj.weight', 'vision_model.encoder.layers.8.self_attn.out_proj.weight', 'vision_model.encoder.layers.3.layer_norm2.weight', 'vision_model.encoder.layers.6.self_attn.v_proj.weight', 'vision_model.encoder.layers.2.self_attn.k_proj.bias', 'vision_model.encoder.layers.9.self_attn.q_proj.bias', 'vision_model.encoder.layers.1.mlp.fc2.bias', 'vision_model.encoder.layers.3.self_attn.q_proj.weight', 'vision_model.enc

In [4]:
inputs = tokenizer(["a photo of a dumb dumb dumb cat", "a photo of a dog today"], padding=True, return_tensors="pt")
inputs

{'input_ids': tensor([[49406,   320,  1125,   539,   320, 12464, 12464, 12464,  2368, 49407],
        [49406,   320,  1125,   539,   320,  1929,   721, 49407, 49407, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])}

In [9]:
print(last_hidden_state.shape)
print(pooled_output.shape)

torch.Size([2, 7, 512])
torch.Size([2, 512])


In [1]:
import torch
from eg3d_diffuser import EG3DConditional
from diffusers import UNet1DModel

diffuser = EG3DConditional(
            sample_size=512,
            in_channels=2,
            out_channels=1,
            layers_per_block=2,  # how many ResNet layers to use per UNet block
            block_out_channels=(128, 128, 256, 256, 512, 512),  # the number of output channes for each UNet block
            time_embed_dim = 512,
            down_block_types=("DownBlock1D", "DownBlock1D", "AttnDownBlock1D", "DownBlock1D", "AttnDownBlock1D", "DownBlock1D"),
            up_block_types=("UpBlock1D", "AttnUpBlock1D", "UpBlock1D", "AttnUpBlock1D", "UpBlock1D", "UpBlock1D"),
        )

2023-03-03 12:41:18.603326: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
diffuser(torch.zeros((1, 2, 512)), 3, torch.zeros((1, 512)))

UNet1DOutput(sample=tensor([[[-0.1018, -0.1018, -0.0593,  0.0032,  0.0856,  0.1181,  0.1004,
           0.1107,  0.1490,  0.1412,  0.0874,  0.0725,  0.0968,  0.1290,
           0.1690,  0.1959,  0.2097,  0.2231,  0.2363,  0.2599,  0.2941,
           0.3170,  0.3286,  0.3381,  0.3456,  0.3495,  0.3498,  0.3435,
           0.3306,  0.3242,  0.3243,  0.3277,  0.3345,  0.3414,  0.3484,
           0.3542,  0.3586,  0.3611,  0.3615,  0.3614,  0.3607,  0.3608,
           0.3617,  0.3626,  0.3637,  0.3635,  0.3620,  0.3624,  0.3649,
           0.3693,  0.3754,  0.3823,  0.3900,  0.4004,  0.4137,  0.4282,
           0.4439,  0.4609,  0.4790,  0.4971,  0.5150,  0.5321,  0.5484,
           0.5608,  0.5695,  0.5746,  0.5761,  0.5781,  0.5805,  0.5819,
           0.5823,  0.5815,  0.5796,  0.5782,  0.5774,  0.5760,  0.5741,
           0.5726,  0.5716,  0.5720,  0.5739,  0.5750,  0.5755,  0.5764,
           0.5775,  0.5789,  0.5806,  0.5833,  0.5871,  0.5931,  0.6014,
           0.6105,  0.6205,  0.